In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
# Database Reading and Manipulation
import pandas as pd

# Linear Algebra
import numpy as np

# Plotting
import matplotlib.pyplot as plt

# Processing results
import json

# Model Selection
from sklearn.model_selection import train_test_split
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV

# Modeling
from sklearn.linear_model import LinearRegression

# Processing
from sklearn.preprocessing import StandardScaler

# Data imputation
from sklearn.impute import SimpleImputer

# Pipeline
from sklearn.pipeline import Pipeline

# Metrics
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import r2_score


# Custom modules
## Model selection
from src.cross_validation.blocking_time_series_split import BlockingTimeSeriesSplit

## Function to print scores
from src.utils.print_scores import print_scores

## Function to calculate score regression metrics
from src.utils.score_regression_metrics import score_regression_metrics

## Function to fill the results metric dict
from src.utils.fill_results_dict import fill_results_dict

<IPython.core.display.Javascript object>

# Functions and definitions

In [3]:
index_to_save = 12

<IPython.core.display.Javascript object>

In [4]:
SEED = 47
METRICS = (
    "neg_root_mean_squared_error",
    "neg_mean_absolute_error",
    "neg_mean_absolute_percentage_error",
    "r2",
)
METRICS_DICT = {
    "neg_root_mean_squared_error": "RMSE",
    "neg_mean_absolute_error": "MAE",
    "neg_mean_absolute_percentage_error": "MAPE",
    "r2": "R2",
}

<IPython.core.display.Javascript object>

## Defining a dataframe structure to save the results

In [5]:
results_to_save = []

results_dict = {
    "Category": "Local Model",
    "Company": "partner_i",
    "Features": "Chemical + Mineralogical + Properties CS Less",
    "Data Shape": None,
    "Timesteps": None,
    "Model": "Linear Regression",
    "Model Params": None,
    "Scaler": "Standard Scaler",
    "Scaler Params": None,
    "Imputer": "Median",
    "Imputer Params": None,
    "Cross Validation": None,
    "Cross Validation Params": np.nan,
    "RMSE Train": np.nan,
    "MAE Train": np.nan,
    "MAPE Train": np.nan,
    "R2 Train": np.nan,
    "RMSE Test": np.nan,
    "MAE Test": np.nan,
    "MAPE Test": np.nan,
    "R2 Test": np.nan,
}

<IPython.core.display.Javascript object>

# Reading the dataset

In [6]:
df = pd.read_csv("../../../../../../data/processed/partner_i-Oficial/cement-shipping.csv")

<IPython.core.display.Javascript object>

## Defining Features

In this set of experiments we keep only chemical and mineralogical features yielded by the same testing method/procedure

In [7]:
df_copy = df.drop(
    [
        # Properties
        "CS3",
        "CS7",
        
        # Removing One-Hot encoding variables
        "Cement_Type",
        
        
        # Chemical Composition
        # Reason: Loss on Ignition is the only feature
        # that belongs to chemical composition in which was 
        # measured by a different method, namely manual
        
    ],
    axis=1,
).copy()


<IPython.core.display.Javascript object>

# 1. Linear Regression

<h2>1. Dataset: df_no_cs</h2> <br>In this dataset the CS1, CS3  and CS7 variables are not considered. Only Chemical and mineralogical features measured by the same method. For this particular dataset, all chemical features, with the exception of LOI were measured by XRF and XRD methods.

In [8]:
y = df_copy.pop("CS28").values
x = df_copy.drop(["Date"], axis=1)
dates = df["Date"].copy()

<IPython.core.display.Javascript object>

## 1.1 Repeated KFold Cross validation

<b>Dataset shape:</b> (1234, 38)<br>
<b>Repeats:</b>10<br>
<b>Splits:</b>10<br>
    1. 10 folds of 123 samples each
    2. 90% train (1111 samples each fold)
    3. 10% test (123 samples each fold)
<b>Total:</b> 100 models<br>

In [9]:
repeats = 3
n_splits = 5
pipeline = Pipeline(
    [
        ("imputer", SimpleImputer(strategy="median")),
        ("transformer", StandardScaler()),
        ("estimator", LinearRegression()),
    ]
)
cv = RepeatedKFold(n_splits=n_splits, n_repeats=repeats, random_state=SEED)
scores = cross_validate(
    pipeline,
    x,
    y,
    scoring=METRICS,
    cv=cv,
    n_jobs=-1,
    return_train_score=True,
)
print("Repeated Cross Validation:")
print(f"Repeats: {repeats}")
print(f"n_splits: {n_splits}")
print()
print_scores(scores, METRICS, METRICS_DICT)

results_dict_copy = results_dict.copy()
results_dict_copy["Cross Validation"] = "Repeated KFold"
results_dict_copy["Cross Validation Params"] = '{"N_Splits": 5, "Repeats": 3}'
results_dict_copy["Data Shape"] = x.shape
df_results = fill_results_dict(results_dict_copy, scores)
results_to_save.append(df_results)

Repeated Cross Validation:
Repeats: 3
n_splits: 5

******
[TRAIN]
******
RMSE: -1.849 (0.031)
MAE: -1.394 (0.020)
MAPE: -0.033 (0.001)
R2: 0.866 (0.005)


******
[TEST]
******
RMSE: -1.952 (0.148)
MAE: -1.464 (0.071)
MAPE: -0.035 (0.002)
R2: 0.849 (0.023)




<IPython.core.display.Javascript object>

In [10]:
pd.concat(results_to_save).reset_index().groupby(
    ["Features", "Model", "Cross Validation"], dropna=False
)[["RMSE Test", "MAE Test", "MAPE Test", "R2 Test"]].agg(
    ["mean", lambda series: pd.Series(series.std(ddof=0), name="std")]
).reset_index().rename(
    columns={"<lambda_0>": "std"}
)

Features              Model  \
                                                                      
0  Chemical + Mineralogical + Properties CS Less  Linear Regression   

  Cross Validation RMSE Test            MAE Test           MAPE Test           \
                        mean       std      mean       std      mean      std   
0   Repeated KFold  1.952452  0.148236  1.463687  0.070749  0.034962  0.00199   

    R2 Test            
       mean       std  
0  0.848642  0.022774

<IPython.core.display.Javascript object>

## 1.2. Blocking Time Series Cross Validation

<b>Dataset shape:</b> (1234, 38)<br>
<b>Splits:</b>5<br>    
    1. 5 folds of 246 samples
    2. 50% train (123 samples each fold)
    3. 50% test (123 samples each fold)
<b>Total:</b> 5 models<br>

In [11]:
n_splits = 5
train_size = 0.8

pipeline = Pipeline(
    [
        ("imputer", SimpleImputer(strategy="median")),
        ("transformer", StandardScaler()),
        ("estimator", LinearRegression()),
    ]
)
cv = BlockingTimeSeriesSplit(n_splits=n_splits, train_size=train_size)
scores = cross_validate(
    pipeline,
    x,
    y,
    scoring=METRICS,
    cv=cv,
    n_jobs=-1,
    return_train_score=True,
)
print("Blocking Time Series Split:")
print(f"Repeats: {repeats}")
print(f"n_splits: {n_splits}")
print()
print_scores(scores, METRICS, METRICS_DICT)

results_dict_copy = results_dict.copy()
results_dict_copy["Cross Validation"] = "Blocking Time Series Split"
results_dict_copy[
    "Cross Validation Params"
] = '{"N_Splits": 5, "Repeats": 1, "train_size": 0.8}'
results_dict_copy["Data Shape"] = x.shape
df_results = fill_results_dict(results_dict_copy, scores)
results_to_save.append(df_results)

Blocking Time Series Split:
Repeats: 3
n_splits: 5

******
[TRAIN]
******
RMSE: -1.532 (0.314)
MAE: -1.187 (0.249)
MAPE: -0.028 (0.006)
R2: 0.907 (0.031)


******
[TEST]
******
RMSE: -1.930 (0.511)
MAE: -1.570 (0.476)
MAPE: -0.037 (0.011)
R2: 0.838 (0.078)




<IPython.core.display.Javascript object>

In [12]:
pd.concat(results_to_save).reset_index().groupby(
    ["Features", "Model", "Cross Validation"], dropna=False
)[["RMSE Test", "MAE Test", "MAPE Test", "R2 Test"]].agg(
    ["mean", lambda series: pd.Series(series.std(ddof=0), name="std")]
).reset_index().rename(
    columns={"<lambda_0>": "std"}
)

Features              Model  \
                                                                      
0  Chemical + Mineralogical + Properties CS Less  Linear Regression   
1  Chemical + Mineralogical + Properties CS Less  Linear Regression   

             Cross Validation RMSE Test            MAE Test            \
                                   mean       std      mean       std   
0  Blocking Time Series Split  1.930302  0.510600  1.569590  0.476164   
1              Repeated KFold  1.952452  0.148236  1.463687  0.070749   

  MAPE Test            R2 Test            
       mean      std      mean       std  
0  0.036869  0.01077  0.837828  0.077886  
1  0.034962  0.00199  0.848642  0.022774

<IPython.core.display.Javascript object>

## 1.3. Time Series Split Cross Validation

The training set has size i * n_samples // (n_splits + 1) + n_samples % (n_splits + 1) in the i th split, with a test set of size n_samples//(n_splits + 1) by default, where n_samples is the number of samples.


<b>Dataset shape:</b> (1234, 38)<br>
<b>Splits:</b>10<br>    
    1. Train: 10 folds of 114, 226, 338, 450, 562, 675, 787, 899, 1011, 1123 samples each fold
    2. Test: 112 samples each fold
<b>Total:</b> 10 models<br>

In [13]:
n_splits = 5
gap = 0
pipeline = Pipeline(
    [
        ("imputer", SimpleImputer(strategy="median")),
        ("transformer", StandardScaler()),
        ("estimator", LinearRegression()),
    ]
)
cv = TimeSeriesSplit(gap=gap, max_train_size=None, n_splits=n_splits, test_size=None)

scores = cross_validate(
    pipeline,
    x,
    y,
    scoring=METRICS,
    cv=cv,
    n_jobs=-1,
    return_train_score=True,
)
print("Time Series Split:")
print(f"Repeats: {repeats}")
print(f"n_splits: {n_splits}")
print()
print_scores(scores, METRICS, METRICS_DICT)

results_dict_copy = results_dict.copy()
results_dict_copy["Cross Validation"] = "Time Series Split"
results_dict_copy["Cross Validation Params"] = '{"N_Splits": 5, "Repeats": 1, "Gap": 0}'
results_dict_copy["Data Shape"] = x.shape
df_results = fill_results_dict(results_dict_copy, scores)
results_to_save.append(df_results)

Time Series Split:
Repeats: 3
n_splits: 5

******
[TRAIN]
******
RMSE: -1.571 (0.252)
MAE: -1.205 (0.172)
MAPE: -0.029 (0.004)
R2: 0.897 (0.029)


******
[TEST]
******
RMSE: -2.227 (0.571)
MAE: -1.722 (0.453)
MAPE: -0.041 (0.011)
R2: 0.800 (0.085)




<IPython.core.display.Javascript object>

In [14]:
pd.concat(results_to_save).groupby(["Features", "Model", "Cross Validation"])[
    ["RMSE Test", "MAE Test", "MAPE Test", "R2 Test"]
].agg(
    ["mean", lambda series: pd.Series(series.std(ddof=0), name="std")]
).reset_index().rename(
    columns={"<lambda_0>": "std"}
)

Features              Model  \
                                                                      
0  Chemical + Mineralogical + Properties CS Less  Linear Regression   
1  Chemical + Mineralogical + Properties CS Less  Linear Regression   
2  Chemical + Mineralogical + Properties CS Less  Linear Regression   

             Cross Validation RMSE Test            MAE Test            \
                                   mean       std      mean       std   
0  Blocking Time Series Split  1.930302  0.510600  1.569590  0.476164   
1              Repeated KFold  1.952452  0.148236  1.463687  0.070749   
2           Time Series Split  2.226611  0.571335  1.721833  0.452641   

  MAPE Test             R2 Test            
       mean       std      mean       std  
0  0.036869  0.010770  0.837828  0.077886  
1  0.034962  0.001990  0.848642  0.022774  
2  0.040999  0.010848  0.799845  0.084622

<IPython.core.display.Javascript object>

## 1.4. Out of time Split Cross Validation

<b>Dataset shape:</b> (1234, 38)<br>
<b>Train size: 80%</b><br>
<b>Test  size: 20%</b>


<b>Splits:</b> 2<br>    
    1. Train: 987
    2. Test: 247
<b>Total:</b> 1 model<br>

In [15]:
test_size = 0.2

x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=test_size, random_state=SEED, shuffle=False
)
pipeline = Pipeline(
    [
        ("imputer", SimpleImputer(strategy="median")),
        ("transformer", StandardScaler()),
        ("estimator", LinearRegression()),
    ]
)

pipeline.fit(x_train, y_train)

y_train_pred = pipeline.predict(x_train)
y_test_pred = pipeline.predict(x_test)

scores = score_regression_metrics(y_train, y_train_pred, y_test, y_test_pred)
print_scores(scores, METRICS, METRICS_DICT)

results_dict_copy = results_dict.copy()
results_dict_copy["Cross Validation"] = "Out of time Split"
results_dict_copy["Cross Validation Params"] = '{"Test Size": 0.2}'
results_dict_copy["Data Shape"] = x.shape
df_results = fill_results_dict(
    results_dict_copy, {key: [value] for key, value in scores.items()}
)
results_to_save.append(df_results)

******
[TRAIN]
******
RMSE: 1.922 (0.000)
MAE: 1.452 (0.000)
MAPE: 0.035 (0.000)
R2: 0.856 (0.000)


******
[TEST]
******
RMSE: 1.631 (0.000)
MAE: 1.228 (0.000)
MAPE: 0.029 (0.000)
R2: 0.892 (0.000)




<IPython.core.display.Javascript object>

In [16]:
pd.concat(results_to_save).groupby(["Features", "Model", "Cross Validation"])[
    ["RMSE Test", "MAE Test", "MAPE Test", "R2 Test"]
].agg(
    ["mean", lambda series: pd.Series(series.std(ddof=0), name="std")]
).reset_index().rename(
    columns={"<lambda_0>": "std"}
)

Features              Model  \
                                                                      
0  Chemical + Mineralogical + Properties CS Less  Linear Regression   
1  Chemical + Mineralogical + Properties CS Less  Linear Regression   
2  Chemical + Mineralogical + Properties CS Less  Linear Regression   
3  Chemical + Mineralogical + Properties CS Less  Linear Regression   

             Cross Validation RMSE Test            MAE Test            \
                                   mean       std      mean       std   
0  Blocking Time Series Split  1.930302  0.510600  1.569590  0.476164   
1           Out of time Split  1.631317  0.000000  1.228395  0.000000   
2              Repeated KFold  1.952452  0.148236  1.463687  0.070749   
3           Time Series Split  2.226611  0.571335  1.721833  0.452641   

  MAPE Test             R2 Test            
       mean       std      mean       std  
0  0.036869  0.010770  0.837828  0.077886  
1  0.028587  0.000000  0.892491  0.000000  
2  0.034962  0.001990  0.848642  0.022774  
3  0.040999  0.010848  0.799845  0.084622

<IPython.core.display.Javascript object>

In [17]:
pipeline.named_steps["estimator"].coef_

array([-1.22391876e-01, -4.23249773e-01,  2.80999158e-02,  1.05197732e+00,
       -1.14096073e+00,  5.89003878e-01, -2.70531054e-01,  8.28391113e-01,
        5.36902531e-01,  1.64505809e-01,  1.14723212e+00,  1.80838893e-01,
        4.25581832e-01, -1.92782038e-01,  4.89784507e-01,  3.61363081e-01,
       -6.38005478e-02, -1.04933849e-03,  2.21439528e-01,  4.20023804e-03,
        1.24643171e-01,  3.73104144e-01,  2.83241041e-01,  8.73018972e-02,
        3.52131351e-01, -9.39581703e-02,  1.36808456e+00,  1.42972995e-01,
        1.98571674e-01, -3.37578531e-01, -4.56617528e-01, -4.33914563e-02,
        5.98907759e-01, -8.85730769e-01, -2.64404576e+00])

<IPython.core.display.Javascript object>

In [18]:
pipeline.named_steps["estimator"].intercept_

43.398173464561005

<IPython.core.display.Javascript object>

In [19]:
coeff = pipeline.named_steps["estimator"].coef_
np.array(x.columns)[coeff == 0]

array([], dtype=object)

<IPython.core.display.Javascript object>

In [20]:
x.drop(np.array(x.columns)[coeff == 0], axis=1).columns

Index(['CaO', 'MgO', 'Na2O', 'Al2O3', 'SiO2', 'SO3', 'K2O', 'Fe2O3',
       'Loss on Ignition', 'Insoluble Residue', 'Total C3S', 'Alpha C2S',
       'Beta C2S', 'Gamma C2S', 'C4AF', 'C3A', 'Cubic C3A', 'Orthorhombic C3A',
       'Free CaO', 'Portlandite', 'Periclase', 'Arcanite', 'Aphthitalite',
       'Gypsum', 'Bassanite', 'Anhydrite', 'Calcite', 'Dolomite', 'Quartz',
       'Blaine', 'Initial setting time', 'Final setting time', 'Density',
       '#200', '#325'],
      dtype='object')

<IPython.core.display.Javascript object>

In [21]:
x.drop(np.array(x.columns)[coeff == 0], axis=1).columns.shape

(35,)

<IPython.core.display.Javascript object>

In [22]:
x.columns.shape

(35,)

<IPython.core.display.Javascript object>

In [23]:
coeffs = pd.DataFrame(
    {col: [c] for col, c in zip(x.columns, coeff)}, index=["Coefficients"]
)

<IPython.core.display.Javascript object>

In [24]:
coeffs.T["Coefficients"].sort_values(ascending=False).to_frame(
    name="Coefficients"
).style.background_gradient(axis=None, vmin=1, vmax=5, cmap="Greens")

,Coefficients
Calcite,1.368085
Total C3S,1.147232
Al2O3,1.051977
Fe2O3,0.828391
Density,0.598908
SO3,0.589004
Loss on Ignition,0.536903
C4AF,0.489785
Beta C2S,0.425582
Arcanite,0.373104


<IPython.core.display.Javascript object>

In [25]:
pd.concat(results_to_save).groupby(
    ["Features", "Model", "Cross Validation", "Cross Validation Params"]
)[["RMSE Test", "MAE Test", "MAPE Test", "R2 Test"]].agg(
    ["mean", lambda series: pd.Series(series.std(ddof=0), name="std")]
).reset_index().rename(
    columns={"<lambda_0>": "std"}
)

Features              Model  \
                                                                      
0  Chemical + Mineralogical + Properties CS Less  Linear Regression   
1  Chemical + Mineralogical + Properties CS Less  Linear Regression   
2  Chemical + Mineralogical + Properties CS Less  Linear Regression   
3  Chemical + Mineralogical + Properties CS Less  Linear Regression   

             Cross Validation  \
                                
0  Blocking Time Series Split   
1           Out of time Split   
2              Repeated KFold   
3           Time Series Split   

                            Cross Validation Params RMSE Test            \
                                                         mean       std   
0  {"N_Splits": 5, "Repeats": 1, "train_size": 0.8}  1.930302  0.510600   
1                                {"Test Size": 0.2}  1.631317  0.000000   
2                     {"N_Splits": 5, "Repeats": 3}  1.952452  0.148236   
3           {"N_Splits": 5, "Repeats": 1, "Gap": 0}  2.226611  0.571335   

   MAE Test           MAPE Test             R2 Test            
       mean       std      mean       std      mean       std  
0  1.569590  0.476164  0.036869  0.010770  0.837828  0.077886  
1  1.228395  0.000000  0.028587  0.000000  0.892491  0.000000  
2  1.463687  0.070749  0.034962  0.001990  0.848642  0.022774  
3  1.721833  0.452641  0.040999  0.010848  0.799845  0.084622

<IPython.core.display.Javascript object>

# Saving the results Dataframe

## Saving the full dataframe

In [26]:
path = (
    "../../../../../../reports/results/local_models/partner_i-oficial/all_cements/full/"
)
filename = f"linear_regression_results_full_{index_to_save}.csv"

pd.concat(results_to_save).to_csv(
    path_or_buf=path + filename,
    mode="w",
    index=False,
    header=True,
)

<IPython.core.display.Javascript object>

## Saving the grouped dataframe

In [27]:
cols_groupby = [
    "Category",
    "Company",
    "Data Shape",
    "Timesteps",
    "Features",
    "Model",
    "Cross Validation",
    "Cross Validation Params",
]

cols_agg = ["RMSE Train", "MAE Train", "MAPE Train", "R2 Train"] + [
    "RMSE Test",
    "MAE Test",
    "MAPE Test",
    "R2 Test",
]

path = "../../../../../../reports/results/local_models/partner_i-oficial/all_cements/grouped/"
filename = f"linear_regression_results_grouped_{index_to_save}.csv"


df_results_to_save = (
    pd.concat(results_to_save)
    .groupby(cols_groupby, dropna=False)[cols_agg]
    .agg(["mean", lambda series: pd.Series(series.std(ddof=0), name="std")])
    .reset_index()
    .rename(columns={"<lambda_0>": "std"})
)

df_results_to_save.to_csv(
    path_or_buf=path + filename,
    mode="w",
    index=False,
    header=True,
)

<IPython.core.display.Javascript object>

In [28]:
df_results_to_save

Category     Company  Data Shape Timesteps  \
                                                   
0  Local Model  partner_i  (1226, 35)       NaN   
1  Local Model  partner_i  (1226, 35)       NaN   
2  Local Model  partner_i  (1226, 35)       NaN   
3  Local Model  partner_i  (1226, 35)       NaN   

                                        Features              Model  \
                                                                      
0  Chemical + Mineralogical + Properties CS Less  Linear Regression   
1  Chemical + Mineralogical + Properties CS Less  Linear Regression   
2  Chemical + Mineralogical + Properties CS Less  Linear Regression   
3  Chemical + Mineralogical + Properties CS Less  Linear Regression   

             Cross Validation  \
                                
0  Blocking Time Series Split   
1           Out of time Split   
2              Repeated KFold   
3           Time Series Split   

                            Cross Validation Params RMSE Train            ...  \
                                                          mean       std  ...   
0  {"N_Splits": 5, "Repeats": 1, "train_size": 0.8}   1.532114  0.313651  ...   
1                                {"Test Size": 0.2}   1.921770  0.000000  ...   
2                     {"N_Splits": 5, "Repeats": 3}   1.848950  0.030944  ...   
3           {"N_Splits": 5, "Repeats": 1, "Gap": 0}   1.571111  0.252125  ...   

   R2 Train           RMSE Test            MAE Test           MAPE Test  \
       mean       std      mean       std      mean       std      mean   
0  0.906798  0.030828  1.930302  0.510600  1.569590  0.476164  0.036869   
1  0.855507  0.000000  1.631317  0.000000  1.228395  0.000000  0.028587   
2  0.865601  0.004649  1.952452  0.148236  1.463687  0.070749  0.034962   
3  0.897254  0.029087  2.226611  0.571335  1.721833  0.452641  0.040999   

              R2 Test            
        std      mean       std  
0  0.010770  0.837828  0.077886  
1  0.000000  0.892491  0.000000  
2  0.001990  0.848642  0.022774  
3  0.010848  0.799845  0.084622  

[4 rows x 24 columns]

<IPython.core.display.Javascript object>